In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Paths
base_dir = 'Data'
train_dir = f'{base_dir}/train'
val_dir = f'{base_dir}/validation'
test_dir = f'{base_dir}/test'

# Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep consistent ordering for accuracy evaluation
)

# Model Definition
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers to transfer learning (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust epochs as needed
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


: 

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_new_image(model, img_path):
    # Load the image and resize to 224x224 pixels
    img = image.load_img(img_path, target_size=(224, 224))
    # Convert the image to a numpy array and normalize pixel values
    img_array = image.img_to_array(img) / 255.0  # Normalize the image (if you used normalization during training)
    # Expand dimensions to match the input shape: (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make a prediction
    prediction = model.predict(img_array)
    print(prediction)
    # Interpret the prediction
    if prediction[0] > 0.5:
        print("Prediction: Spam Image")
    else:
        print("Prediction: Non-Spam Image")

# Example usage
predict_new_image(model, 'WIN_20230404_11_45_29_Pro.jpg')


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
# Evaluate the model on the test set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Test Accuracy: {val_accuracy:.2f}')


In [ ]:
# Evaluate the model on the test set
train_loss, train_accuracy = model.evaluate(train_generator)
print(f'Test Accuracy: {train_accuracy:.2f}')


In [ ]:
model.save('model1.h5')

In [ ]:
train_generator_2 = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep consistent ordering for accuracy evaluation
)

val_generator_2 = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep consistent ordering for accuracy evaluation
)

In [ ]:
def evaluate_model_with_threshold(model, generator, threshold=0.5):
    predictions = model.predict(generator)
    y_true = generator.classes
    y_pred = (predictions > threshold).astype(int).flatten()

    # Calculate accuracy
    correct_predictions = np.sum(y_true == y_pred)
    accuracy = correct_predictions / len(y_true)
    return accuracy

# Calculate and print accuracy
accuracy = evaluate_model_with_threshold(model, train_generator_2)
print(f"Model accuracy on the training dataset with threshold {0.7}: {accuracy:.2%}")